In [2]:
import os
import pandas as pd 
from bs4 import BeautifulSoup
from lxml import etree
from os import listdir
from os.path import isfile, join

In [3]:
df = pd.DataFrame()
df_lost_time = pd.DataFrame()
print("this is the main df")
print(df)
print("this is the df for lost time")
print(df_lost_time)

this is the main df
Empty DataFrame
Columns: []
Index: []
this is the df for lost time
Empty DataFrame
Columns: []
Index: []


In [4]:
def extractdata(df, filename):

    #opening the the html file
    with open(filename, 'r') as f:

        contents = f.read()
        #this is lxml method to read
        #soup = BeautifulSoup(contents, 'lxml')
        #this is beautiful soup html parser 
        soup = BeautifulSoup(contents, "html.parser")
        dom = etree.HTML(str(soup))

        print("File: " + soup.title.text)

        #assign soup items in to table and rows rows is table body and table rows 
        table = soup.find("table", {"id": "AutoNumber2"})     
        #printing the table
        #print(table)
        rows = table.find("tbody").find_all("tr")
        #neg_length_rows = 0 - len(rows)
        #print("length of rows is:" ,neg_length_rows)


        for row in rows:
            columns = row.find_all("td")
            print("lenght of the column is" , len(columns))
        #for i in range(neg_length_rows,-1):
        #   columns = rows[i].find_all("td")
        #    columns[-1].get_text()
        #    if columns == "LT Summary":
        #        break

            if len(columns) == 16 or len(columns) == 18 or len(columns) == 23:
                continue
            else:
                #iterating over the rows from table
                #this will grab all the columns for each row 
                #the number of columns per each row is
                # 16 if there is no NPT
                # 18 for upper table
                # 11 for lost time table
                # there is an item with 23 elements it is in the middle
                
                #print(getTextFromTag(columns, 18))
                #print(getTextFromTag(columns, lastrow_index)
                try:
                    date = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[2]/table/tbody/tr[3]/td')[0].text
                    wellno = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[3]/table[1]/tbody/tr[2]/td')[0].text
                    rigname = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[4]/table/tbody/tr[2]/td')[0].text
                    Date_and_time = columns[-11].get_text()
                    Time_in_Hrs = columns[-10].get_text()
                    cum_Hrs = columns[-9].get_text()
                    LT_ID = columns[-8].get_text()
                    Parent_LT_ID = columns[-7].get_text()
                    LT_type = columns[-6].get_text()
                    cause = columns[-5].get_text()      
                    object = columns[-4].get_text()
                    company = columns[-3].get_text()
                    depth = columns[-2].get_text()
                    LT_summary = columns[-1].get_text()
                except:
                    date = ""
                    wellno = ""
                    rigname = ""
                    Date_and_time = ""
                    Time_in_Hrs = ""
                    cum_Hrs = ""
                    LT_ID = ""
                    Parent_LT_ID = ""
                    LT_type = ""
                    cause = ""      
                    object = ""
                    company = ""
                    depth = ""
                    LT_summary = ""
                #checking for extra line of information about the depth 
                #and company this will ignore that line. it is un necessary 
                # if company == "Depth":
                #     continue
                    #LT_depth == company
                data = {
                    #this is is to capture lost time
                    "Report Date": date,
                    "Rig Name": rigname,
                    "Well No": wellno, 
                    "Date and Time": Date_and_time,
                    "Time in Hrs": Time_in_Hrs,
                    "Cummulative Hrs" : cum_Hrs,
                    "LT ID": LT_ID,
                    "Parent LT ID":Parent_LT_ID,
                    "LT Type": LT_type,
                    "Cause" : cause,
                    "Object": object,
                    "Company Name": company, 
                    "Depth": depth,
                    "LT Summary": LT_summary,
                    }
                
                df = df.append(data, ignore_index = True)
                
    return df

In [5]:
#this takes in df and morning reports folder
#it goes through all files in the folder 
#returns data frame 
def readReportsPerDay(df, morningReportFolder):

    mypath = morningReportFolder + "/reports"
    #mypath += "/reports"
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    count = 1

    for filename in onlyfiles:

        if "DS_Store" in filename:
            continue
        filename = mypath + "/" + filename
        print(str(count) + " " + filename)

        df = extractdata(df, filename)
        count += 1

    return df

In [6]:
#this funtion will call the reports per day function 
#this is to be used for running the program in folder
#it goes thru the all morning report folders per day inside specific folder 
def runInRunningFolder(df):

    for morningReportFolder in listdir(os.getcwd()):

        if ".ipynb" in morningReportFolder or ".git" in morningReportFolder or ".csv" in morningReportFolder or "DS_Store" in morningReportFolder or ".xlsx" in morningReportFolder or ".json" in morningReportFolder or ".pbix" in morningReportFolder or ".zip" in morningReportFolder:
            continue

        print(morningReportFolder + "\n")

        df = readReportsPerDay(df, morningReportFolder)

    return df

In [7]:
print(df)

Empty DataFrame
Columns: []
Index: []


In [8]:
#this is where we run the code for each item in directory 
#this will list curent working directory and rung th code 

print(os.getcwd())


df = runInRunningFolder(df)

print(df)

c:\Users\ygebremedhin\OneDrive - Schlumberger\GitHub\Lost_Time_table_data_Akoji\Running folder\Lost Time HTML files
MR 08-30-2022

1 MR 08-30-2022/reports/095TE-MNIF-126.html
File: 095TE
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 23
lenght of the column is 11
lenght of the column is 11
2 MR 08-30-2022/reports/ADC-17-ZULF-1385.html
File: ADC-17
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 23
lenght of the column is 11
lenght of the column is 11
3 MR 08-30-2022/reports/ADC-35-MAQL-5.html
File: ADC-35
lenght of the column is 18
lenght of the column is 18
lenght of the column is 18
lenght of the column is 23
lenght of the column is 11
lenght of the column is 11
4 MR 08-30-2022/reports/ADC-55L-UTMN-3171.html
Fil

In [11]:
print(df)

    Cause Company Name Cummulative Hrs       Date and Time   Depth    LT ID  \
0   Cause      Resp Co        Cum. Hrs          From  - To   Depth    LT ID   
1    FRFM          ARM              65  28 Aug 1030 - 0500    6100   901348   
2   Cause      Resp Co        Cum. Hrs          From  - To   Depth    LT ID   
3    FPMP          RIG              .5  30 Aug 0245 - 0315    1470   901503   
4   Cause      Resp Co        Cum. Hrs          From  - To   Depth    LT ID   
5     FOT          HCB            28.5  29 Aug 0030 - 0500    5915   901459   
6    ORDS          TK3              15  30 Aug 1400 - 0500   10968   901520   
7   Cause      Resp Co        Cum. Hrs          From  - To   Depth    LT ID   
8     FOT          RIG             156  23 Aug 0200 - 1400   10968   901375   
9    ORDS          TK3              15  30 Aug 1400 - 0500   10968   901520   
10  Cause      Resp Co        Cum. Hrs          From  - To   Depth    LT ID   
11    FOT          RIG             1.5  30 Aug 0530 

In [12]:
df.to_excel("LOST TIME TEST.xlsx")

In [30]:
def getTextFromTag(columns, index):
    columnText = ""
    try:
        columnText = columns[index].get_text()
    except:
        pass
    return columnText

#this is extract data function
#this function is callled in reports per day function to extract data 
#it reads one file and adds to data frame
#it takes data frame and returns data frame updated per files 
def extractdata(df, filename):

    with open(filename, 'r') as f:

        contents = f.read()
        #this is lxml method to read
        #soup = BeautifulSoup(contents, 'lxml')

        #this is beautiful soup html parser 
        soup = BeautifulSoup(contents, "html.parser")
        dom = etree.HTML(str(soup))

        print("File: " + soup.title.text)
        
        try:
            linerSize = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td[1]')[0].text
            date = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[2]/table/tbody/tr[3]/td')[0].text
            wellno = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[3]/table[1]/tbody/tr[2]/td')[0].text
            rigname = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[4]/table/tbody/tr[2]/td')[0].text
        except:
            linerSize = ""
            date = ""
            wellno = ""
            rigname = ""
        
        if linerSize is None:
            linerSize = "None"
        #assign soup items in to table and rows rows is table body and table rows 
        table = soup.find("table", {"id": "AutoNumber2"})     
        #printing the table
        #print(table)
                
        rows = table.find("tbody").find_all("tr")

        for row in rows:
            
            #this will grab all the columns for each row 
            columns = row.find_all("td")
            print(len(columns))
            lateral = getTextFromTag(columns, 4)
            holesize = getTextFromTag(columns, 5)
            majoroperation = getTextFromTag(columns, 7)
            action = getTextFromTag(columns, 8)
            operation = getTextFromTag(columns, 9)
            company = getTextFromTag(columns, 10)
            holeDepthStart = getTextFromTag(columns, 11)
            holeDepthEnd = getTextFromTag(columns, 12)
            eventDepthStart = getTextFromTag(columns, 13)
            eventDepthEnd = getTextFromTag(columns, 14)
            if len(columns) == 16:
                SummaryOfOperation = getTextFromTag(columns, 15)
            else: 
                SummaryOfOperation = getTextFromTag(columns, 17)
            #checking for lost time
                if company == "LT Summary":
                    continue
            data = {
                "Date": date,
                "Well No": wellno,
                "Liner Size": linerSize,
                "Operation": operation,
                "Rig Name": rigname,
                "lateral":lateral,
                "holesize":holesize,
                "majoroperation": majoroperation,
                "action": action,
                "Company Name": company,              
                "Hole Depth Start" : holeDepthStart,
                "Hole Depth End" : holeDepthEnd,
                "Event Depth Start" : eventDepthStart,
                "Event Depth End" : eventDepthEnd,
                "Summarry Of Operation" : SummaryOfOperation       
            }
            
            df = df.append(data, ignore_index = True)

    return df

Empty DataFrame
Columns: []
Index: []
